## Import Section

In [15]:
import numpy as np
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn import preprocessing
from sklearn import utils
from sklearn.metrics import scorer
from sklearn.metrics import accuracy_score

### Database connection

In [16]:
conn_string = "host='localhost' dbname='stockmarket' user='postgres' password='5265104d'"
conn = psycopg2.connect(conn_string)

## Read dataset

In [17]:
df = pd.read_sql_query(
    '''select i.date ,i.open ,i.high ,i.low ,i.close ,i.adj_close ,i.volume ,c.name
    from dataset_information i 
    join dataset_company c
    on i.company_id=c.id
    where company_id = {}'''.format(3)  , conn, index_col='date')

## Calculate bolingerband 

In [18]:
df["20d"] = np.round(df["close"].rolling(window = 20, center = False).mean(), 2)
# 2. Compute rolling standard deviation
apple_rstd = np.round(df['close'].rolling(window = 20, center = False).std(), 2)

# 3. Compute upper and lower bands
df['upperband'] = df['20d'] + 2 * apple_rstd
df['lowerband'] = df['20d'] - 2 * apple_rstd

## prepairing dataframe with label

In [19]:
#feature_cols = ['high-upperband', 'low-lowerband','dicesion']

df['high-upperband'] = df['high']-df['upperband']
df['low-lowerband'] =df['low']-df['lowerband']
df['decision'] = np.where(df['high-upperband'] > 0.0 , 'Sell',np.where( df['low-lowerband'] < 0.0,'Buy','Buy & Sell'))
df = df.dropna()
print(df.tail())

                  open        high         low       close   adj_close  \
date                                                                     
2018-03-07  174.940002  175.850006  174.270004  175.029999  175.029999   
2018-03-08  175.479996  177.119995  175.070007  176.940002  176.940002   
2018-03-09  177.960007  180.000000  177.389999  179.979996  179.979996   
2018-03-12  180.289993  182.389999  180.210007  181.720001  181.720001   
2018-03-13  182.589996  183.500000  179.240005  179.970001  179.970001   

                volume  name     20d  upperband  lowerband  high-upperband  \
date                                                                         
2018-03-07  31703500.0  AAPL  170.85     185.69     156.01       -9.839994   
2018-03-08  23774100.0  AAPL  171.72     185.78     157.66       -8.660005   
2018-03-09  32185200.0  AAPL  172.97     185.13     160.81       -5.130000   
2018-03-12  32207100.0  AAPL  174.23     184.19     164.27       -1.800001   
2018-03-13  3

## Prepairing Feature columns and label columns

In [20]:
feature_cols = ['close','20d','upperband','lowerband']
label_cols = ['decision']
x = df[feature_cols]
y = df[label_cols]
npY = np.array(y)

In [21]:
def extractBolingerFeatures(df):
    feature_cols = ['adj_close','20d','upperband','lowerband']
    label_cols = ['decision']
    x = df[feature_cols]
    y = df[label_cols]
    npY = np.array(y)
    return (x,y,npY)

## Split dataset into test and train parts

In [22]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=1)

## prepairing Nayiv Bayes model

In [23]:
gnb = GaussianNB()

In [24]:
def ApplyGuassianNbModel(fetures,label,predictList):
    gnb = GaussianNB()
    gnb.fit(fetures,label)
    y_predict = str(gnb.predict(predictList))
    return y_predict

### test ApplyGuassianNbModel

In [25]:
adj_close = 175.029999
midband = 170.85
upperband = 185.69
lowerband = 156.01
predictCol = [[175.02999,170.85,185.69,156.01]]
result = ApplyKnnModel(x,y,predictCol)
print(result)

['Buy & Sell']


C:\Users\moham\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### test ApplyGuassianNbModel

In [26]:
adj_close = 175.029999
midband = 170.85
upperband = 185.69
lowerband = 156.01
predictCol = [[175.02999,170.85,185.69,156.01]]
result = ApplyGuassianNbModel(x,y,predictCol)
print(result)

['Buy & Sell']


C:\Users\moham\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## basic Guasian Nayiv test

In [27]:
gnb.fit(X_train,y_train)
y_predict = gnb.predict(X_test)
print(float(accuracy_score(y_test, y_predict)) *100)

78.50931677018633


C:\Users\moham\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
